# DimensionReducer Examples

Этот notebook демонстрирует основные возможности модуля DimensionReducer для анализа и сжатия SEO-характеристик.

In [ ]:
import sys
sys.path.append('..')

import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from models.dim_reducer.model import DimensionReducer
from models.dim_reducer.inference import DimReducerInference
from models.dim_reducer.features import SEOFeaturesExtractor
from common.config.dim_reducer_config import DimReducerConfig

## 1. Подготовка данных

Создадим пример данных для демонстрации возможностей модуля.

In [ ]:
# Пример текстов
sample_texts = [
    "This is a sample text about SEO optimization and content marketing strategies.",
    "Another example discussing digital marketing and website analytics.",
    "Third sample text about search engine optimization techniques."
]

# Пример HTML
sample_html = [
    """
    <html>
        <head>
            <title>SEO Sample</title>
            <meta name="description" content="Sample description">
        </head>
        <body>
            <h1>Main Header</h1>
            <p>Content here</p>
        </body>
    </html>
    """
    for _ in range(3)
]

# Создаем DataFrame
df = pd.DataFrame({
    'text': sample_texts,
    'html': sample_html
})

df.head()

## 2. Извлечение SEO-характеристик

Демонстрация работы с SEOFeaturesExtractor.

In [ ]:
# Инициализация экстрактора
extractor = SEOFeaturesExtractor()

# Извлечение характеристик для первого текста
features = extractor.extract_all_features(
    df['text'].iloc[0],
    df['html'].iloc[0]
)

# Вывод основных метрик
print("Basic Metrics:")
metrics = {
    k: v for k, v in features.items()
    if isinstance(v, (int, float))
}
pd.Series(metrics)

### Визуализация важности ключевых слов

In [ ]:
# Построение графика частот слов
plt.figure(figsize=(12, 5))
word_freq = pd.Series(features['word_frequencies'])
word_freq.sort_values(ascending=True).plot(kind='barh')
plt.title('Word Frequencies')
plt.xlabel('Frequency')
plt.tight_layout()

## 3. Работа с моделью

Демонстрация обучения и применения модели.

In [ ]:
# Конфигурация
config = DimReducerConfig(
    input_dim=512,
    hidden_dim=256,
    latent_dim=128
)

# Создание модели
model = DimensionReducer(config)

# Создание тестовых данных
test_input = torch.randn(5, config.input_dim)
outputs = model(test_input)

print(f"Input shape: {test_input.shape}")
print(f"Latent shape: {outputs['latent'].shape}")
print(f"Reconstructed shape: {outputs['reconstructed'].shape}")

### Визуализация сжатого пространства

In [ ]:
# Создаем тепловую карту латентного пространства
plt.figure(figsize=(10, 4))
sns.heatmap(
    outputs['latent'].detach().numpy(),
    cmap='viridis',
    xticklabels=10,
    yticklabels=True
)
plt.title('Latent Space Heatmap')
plt.xlabel('Latent Dimension')
plt.ylabel('Sample')
plt.tight_layout()

## 4. Инференс

Демонстрация применения обученной модели.

In [ ]:
# Сохранение модели
torch.save(model.state_dict(), 'temp_model.pt')

# Инициализация инференса
inference = DimReducerInference('temp_model.pt', config)

# Обработка текста
results = inference.process_text(df['text'].iloc[0])

# Визуализация важности признаков
plt.figure(figsize=(10, 4))
plt.plot(results['feature_importance'])
plt.title('Feature Importance')
plt.xlabel('Feature Index')
plt.ylabel('Importance Score')
plt.grid(True)
plt.tight_layout()

## 5. Пакетная обработка

Демонстрация обработки нескольких текстов.

In [ ]:
# Пакетная обработка всех текстов
batch_results = inference.batch_process(
    test_input,
    batch_size=2
)

# Визуализация результатов
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.imshow(batch_results['latent_features'], aspect='auto')
plt.title('Latent Features')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(batch_results['feature_importance'])
plt.title('Batch Feature Importance')
plt.grid(True)

plt.tight_layout()